In [7]:
import sys
from IPython.display import clear_output

sys.path.append("./matlab/myBTD2/myBTD2/for_redistribution_files_only")

import os

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/"

import myBTD2
import numpy as np
import warnings
from tqdm import tqdm
import matlab
from scipy.linalg import hankel


warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)


def hankelization(recording):
    _, n_lead = recording.shape

    T = []
    for i in range(0, n_lead):
        vec = recording[:, i]
        vec_filtered_hankel = hankel(vec)
        T.append(vec_filtered_hankel)

    T = np.array(T)
    T = matlab.double(T.tolist(), size=T.shape)

    return T


def get_btd(recording):
    btd = myBTD2.initialize()
    _, output = btd.myBTD2(
        5,
        matlab.double([17], size=(1, 1)),
        matlab.double([300], size=(1, 1)),
        matlab.double([1.0e-5], size=(1, 1)),
        matlab.double([1.0e-5], size=(1, 1)),
        matlab.double([False], size=(1, 1)),
        hankelization(recording),
        nargout=2,
    )
    btd.terminate()

    keys_to_extract = [
        "iter",
        "fval",
        "relfval",
        "relstep",
        "delta",
        "rho",
        "relerr",
    ]

    object_output = [output[key] for key in keys_to_extract]

    return object_output

In [8]:
import re

alias_db = "AFDB"

source_files = [filename for filename in os.listdir(f'../extract_chunks/{alias_db}') if re.search(r'_0', filename)]
dest_files = os.listdir(f'./{alias_db}')
t_results = sorted([file for file in source_files if file not in dest_files])

In [9]:
t_results

['05261_0_178_result.npy',
 '06426_0_200_result.npy',
 '06453_0_84_result.npy',
 '06995_0_200_result.npy',
 '07162_0_200_result.npy',
 '07859_0_200_result.npy',
 '07879_0_200_result.npy',
 '07910_0_200_result.npy',
 '08215_0_200_result.npy',
 '08219_0_200_result.npy',
 '08378_0_200_result.npy',
 '08405_0_200_result.npy',
 '08434_0_200_result.npy',
 '08455_0_200_result.npy']

In [10]:
import numpy as np

if not os.path.exists(alias_db):
    os.makedirs(alias_db)

for filename in t_results:
    samples = np.load(f"../extract_chunks/{alias_db}/{filename}", allow_pickle=True)

    recording_result = np.empty((0, 57))

    for sample in tqdm(samples):
        rri_segment, recording = sample

        rri_histogram, _ = np.histogram(a=rri_segment, range=(0, 2500), bins=50)

        if rri_histogram.shape[0] == 0:
            continue

        btd = get_btd(recording)
        clear_output(wait=True)

        combine = np.hstack((rri_histogram, np.array(btd)))

        recording_result = np.vstack((recording_result, combine))

    np.save(
        file=f'./{alias_db}/{filename}',
        arr=recording_result,
    )

 81%|████████▏ | 145/178 [1:19:52<19:42, 35.82s/it]

> In ll1_core/state (line 237)
In ll1_core (line 191)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In myBTD2 (line 9)

> In ll1_core/state (line 237